In [11]:
import requests
import json
import pandas


class scraper():
    """
    Use the API to scrape dynalist to get the dom
    """

    def __init__(self) -> None:
        self.token = json.load(open('../../api_token.json')).get("token")
        self.file_contents: dict[str: str] = {}
        self.file_questions_map: dict[str: dict[str: str]] = {}

    def get_all_files(self) -> None:
        """
        grabs all folders and files from within dynalist by querying the API
        Stores information about the files in self.file_info
        Stores dictionary of file id: file name, to be able to fetch the full doc
        """

        response = requests.post(
            "https://dynalist.io/api/v1/file/list", json.dumps({'token': self.token})).text

        # Now convert the text representation of a dictionary, into a dictionary
        response = eval(response.replace(
            'false', 'False').replace('true', 'True'))

        # files_info has form: name (type): number
        files_info: list[str] = []
        filenumber_id_map: dict[str: str] = {}

        current_file = 1
        for file in response['files']:
            title = file['title']
            file_type = file['type']
            id = file['id']

            files_info.append(f"{title} ({file_type}): {current_file}")
            filenumber_id_map[current_file] = id

            current_file += 1

        self.files = files_info
        self.filenumber_id_map = filenumber_id_map

    def get_file_contents(self, filenumber: int) -> None:

        file_id = self.filenumber_id_map[filenumber]
        args = {
            'token': self.token,
            'file_id': file_id
        }

        response = requests.post(
            "https://dynalist.io/api/v1/doc/read", json.dumps(args)
        )
        self.file_contents[file_id] = eval(response.text.replace(
            'false', 'False').replace('true', 'True'))

    def save_file_contents(self, filenumber: int) -> None:

        file_id = self.filenumber_id_map[filenumber]
        file_contents = self.file_contents[file_id]

        question_answer_map: dict[str: str] = {}

        nodes = file_contents['nodes']

        for node in nodes:
            node_text = node['content']
            questions = [
                'question:',
                'questions:',
                'question',
                'questions',
            ]
            if node_text.lower().strip() in questions:
                main_child_nodes = node['children']
                for child_id in main_child_nodes:
                    # Now need to search for the node with this id
                    question = main_child_node['content']
                    answer = ''
                    question_child_nodes = main_child_node['children']
                    for question_child_node in question_child_nodes:
                        answer += question_child_node['content']
                        answer += '<br/>'

                    question_answer_map[question] = answer
        
        self.file_questions_map[file_id] = question_answer_map

    def update_file_colors(self, filenumber: int) -> None:

        file_id = self.filenumber_id_map[filenumber]
        file_contents = self.file_contents[file_id]


In [12]:
a = scraper()
a.get_all_files()

In [13]:
a.files

['Untitled (folder): 1',
 'Project priority list (document): 2',
 'Flask with python (document): 3',
 'Udemy (folder): 4',
 'Flask with python (folder): 5']

In [14]:
a.get_file_contents(3)

In [15]:
a.save_file_contents(3)

TypeError: string indices must be integers